In [26]:
import time
import requests
import numpy as np
import pandas as pd
from lxml import html
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager


In [115]:
ESWD_URL = 'https://eswd.eu/cgi-bin/eswd.cgi'
ESWD_TORNADO_XPATH = '//*[@name="TORNADO"]'
YEARS_TO_COUNT = ['1994', '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']
MONTHS_TO_COUNT = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
START_DATE_XPATH = '//*[@id="start_date"]'
END_DATE_XPATH = '//*[@id="end_date"]'
FIND_REPORTS_COUNT_XPATH = "//p[contains(text(),'selected reports')] | //p[contains(text(),'no reports')]"
SUBMIT_XPATH = '(//*[@value="submit query"])[2]'



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [/Users/elad/.wdm/drivers/chromedriver/mac64/96.0.4664.45/chromedriver] found in cache
/var/folders/gm/0p8xp0qn071cptf2y2k3ly7h0000gn/T/ipykernel_28315/3041211002.py:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  return webdriver.Chrome(ChromeDriverManager().install())
/var/folders/gm/0p8xp0qn071cptf2y2k3ly7h0000gn/T/ipykernel_28315/3041211002.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(ESWD_TORNADO_XPATH).click()
/var/folders/gm/0p8xp0qn071cptf2y2k3ly7h0000gn/T/ipykernel_28315/3041211002.py:30: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath(START_DATE_XPATH).clear()
/var/folders/gm/0p8xp0qn071cptf2y2k3ly7h0000gn/T/ipykernel_283

In [ ]:
# auxilary functions for creating the web driver and closing it
def create_web_driver():
    return webdriver.Chrome(ChromeDriverManager().install())

def start_web_driver(driver):
    driver.maximize_window()
    driver.get(ESWD_URL)
    driver.find_element_by_xpath(ESWD_TORNADO_XPATH).click()

def close_web_driver(driver):
    driver.close()

def log(message,filename):
    with open(f'{filename}', 'a') as f:
        f.write(message + '\n')


In [ ]:
# functions to parse the relevant date and input it in the relevant fields of the web driver received by input_date
def get_date(curr_year, curr_month):
    if curr_month == '02':
        day = '28'
    elif curr_month == '04' or curr_month == '06' or curr_month == '09' or curr_month == '11':
        day = '30'
    else:
        day = '31'
    return f'{day}-{curr_month}-{curr_year}', f'01-{curr_month}-{curr_year}'

def input_date(driver, start_date, end_date):
    driver.find_element_by_xpath(START_DATE_XPATH).clear()
    time.sleep(0.5)
    driver.find_element_by_xpath(START_DATE_XPATH).send_keys(start_date)
    driver.find_element_by_xpath(END_DATE_XPATH).clear()
    time.sleep(0.5)
    driver.find_element_by_xpath(END_DATE_XPATH).send_keys(end_date)


In [ ]:
# function to accumulate the data from the site in lists to prepare for dataframe creation using other auxilary functions
def get_data_from_page(driver,year,district, city, country, coordinates, date, times, scale,length,years):
    base_xpath ="//td[@class='base_info']/p[b and not(@class='smallgray')]"
    district_xpath = "/b[1]"
    bold_tags_xpath = "/b"
    country_xpath = "/b[2]"
    date_xpath = "/b[3]"
    time_xpath = "/b[4]"
    scale_xpath_1 = "(//p[@class='TORNADO detail_info_entry'])"
    scale_xpath_2 = "/b[contains(text(),'F')]" 
    scale_xpath_3 = "((//p[@class='TORNADO'][contains(b,'tornado')])"
    scale_xpath_4 = "/b[text()='tornado'])"
    reports_count = len(driver.find_elements_by_xpath(base_xpath))
    len_elements = driver.find_elements_by_xpath(scale_xpath_1)
    parent_items= driver.find_elements_by_xpath(base_xpath)
    for i in range(reports_count):
        years.append(year)
        district = get_elements_from_list(driver,district_xpath,base_xpath,district,i)
        child_item = driver.find_elements_by_xpath((f'({base_xpath})[{i+1}]{bold_tags_xpath}'))
        child_item_text = get_text_from_element_list(child_item)
        city= get_city_list(child_item_text,parent_items[i].text,city,0)
        country = get_elements_from_list(driver,country_xpath,base_xpath,country,i)
        coordinates =  get_coordinates_list(child_item_text,parent_items[i].text,coordinates,1)
        date = get_elements_from_list(driver,date_xpath,base_xpath,date,i)
        times = get_elements_from_list(driver,time_xpath,base_xpath,times,i)
        scale_elements_list = driver.find_elements_by_xpath((f'{scale_xpath_1}[{i+1}]{scale_xpath_2} | {scale_xpath_3}[{i+1}]{scale_xpath_4}'))
        scale = get_scale_list(scale,scale_elements_list)
        length = get_len_list(child_item_text,parent_items[i].text,length,1,len_elements[i])
    return district, city, country, coordinates, date, times, scale,length,years


In [ ]:
# auxilary functions in charge of initial parsing and cleaning of the data captured by XPATH
def replace_child_text(child_text,field):
    for word in child_text.split(' '):
        field = field.replace('\n','').replace(word,'',1).lstrip()
    return field

def checks_split_position(field,selector):
    if field[0] == '(':
        selector = selector + 1
    return selector

def checks_digit(field,selector):
    return field.split('(')[selector][0].isdigit()

def checks_len(field):
    return '<' in field
    
def get_text_from_element_list(elem_list):
    text = ''
    for elem in elem_list:
        text += elem.text + ' '
    return text


In [ ]:
# auxilary functions to clean the data further and appending it to the relevant lists
def append_coordinates_list(field,curr_list,position):
    if checks_len(field):
        curr_list.append(field.split('(')[position].split(')')[0].rstrip())
    else:
        curr_list.append(field.split('(')[position].rstrip().replace(')',''))
    return curr_list

def get_coordinates_list(child_text,field,curr_list,selector):
    field = replace_child_text(child_text,field)
    position = checks_split_position(field, selector)
    if checks_digit(field,position):
            curr_list = append_coordinates_list(field,curr_list,position)
    else:
        if checks_digit(field,position-1):
            curr_list = append_coordinates_list(field,curr_list,position-1)
        else:
            curr_list.append(np.nan)
    return curr_list


In [ ]:
# auxilary functions to clean the data further and appending it to the relevant lists
'''if the size of the elements list is 1 it gets only "tornado" dummy text, so there is no scale in the text, otherwise there is scale 
in the text so take the relevant F scale'''
def get_scale_list(scale,scale_elements_list):
    if len(scale_elements_list) == 1:
        scale.append(np.nan)
    else:
        for element in scale_elements_list:
            if 'F' in element.text:
                scale.append(element.text)
    return scale
'''if there's a field in the right textbox with specified path length get the number presented in it, otherwise check if the first char is a digit, if so check if the field on the left 
    has the "<" char, which indicates length, if so take it, otherwise input nan in the list'''
def get_len_list(child_text,field,curr_list,selector,element):
    if 'path length:' in element.text:
        curr_list.append(element.text.split('path length:')[1].split('k')[0].strip()) 
    else:
        field = replace_child_text(child_text,field)
        if checks_digit(field,selector):
            if checks_len(field):
                curr_list.append(field.split('<')[1].split('k')[0].strip()) 
            else:
                curr_list.append(np.nan)
        else:
            curr_list.append(np.nan)
    return curr_list
'''if the first character is a digit put nan, otherwise take the city name'''
def get_city_list(child_text,field,curr_list,selector):
    field = replace_child_text(child_text,field)
    position = checks_split_position(field, selector)
    if checks_digit(field,position):
        curr_list.append(np.nan)
    else:
        curr_list.append(field.split('(')[position].rstrip().replace(')',''))
    return curr_list

In [ ]:
#auxilary functions to input bold text to the relevant lists and create a df using the lists
def get_elements_from_list(driver,curr_xpath,base_xpath,curr_list,i):
    curr_list.append(driver.find_elements_by_xpath((base_xpath+curr_xpath))[i].text)
    return curr_list

def get_df_from_lists(district, city, country, coordinates, date, times, scale,length,year):
    df = pd.DataFrame({'district':district,'city':city,'country':country,'coordinates':coordinates,'date':date,'time':times,'scale':scale,'length (km)':length,'year':year})
    return df

In [ ]:
# main function to crawl the data from the website and create a df
driver = create_web_driver()
start_web_driver(driver)
district, city, country, coordinates, date, times, scale,length,years = [], [], [], [], [], [], [], [],[]
for year in YEARS_TO_COUNT:
    for month in MONTHS_TO_COUNT:
        end_date, start_date = get_date(year, month)
        input_date(driver, start_date, end_date)
        driver.find_element_by_xpath(SUBMIT_XPATH).click()
        reports_amount = driver.find_element_by_xpath(FIND_REPORTS_COUNT_XPATH).text
        log(f'{year}-{month}-{reports_amount}', 'amount_of_reports.log')
        if "no report" in reports_amount:
            continue
        time.sleep(2)
        district, city, country, coordinates, date, times, scale,length,years = get_data_from_page(driver,year,district, city, country, coordinates, date, times, scale,length,years)
        edf = get_df_from_lists(district, city, country, coordinates, date, times, scale,length,years)
edf.to_csv(f'ESWD_data.csv')
close_web_driver(driver)
